In [1]:
# Importing packages
import numpy as np
import matplotlib
matplotlib.use('tkagg')
import matplotlib.pyplot as plt
import random
from scipy.integrate import odeint

In [2]:
# Define initial states
E = [5]
L = [0]
P = [0]
A = [0]

# Initial time defined
t = [0] # starting from day 1

#Time to end the simulation
tend = 15 # days

################################################################
#Model parameters
a = 8.9231 
b1 = -8.7635
b2 = b1

# Defining temperature function
def temp(t): 
    temp = a + b1*np.sin(2 * np.pi * t[-1]/365) + b2*np.cos(2 * np.pi * t[-1]/365) + np.random.normal(loc = 0.0, scale = 30.0, size = None) 
    return(temp)


#Parameter values
# Fit scalar in gentrophic cycle rate function
alpha = 250.0257252
beta = 34.0632871
gamma = 5.3371833 

#genotrophic cycle as a function of time
def gen_cycle(t):
    func = gamma*np.exp(-(temp(t)-beta)**2/alpha)
    return(func)

# Fit scalars in egg development rate function
alpha_e = 1317.666814
beta_e = 75.098187
gamma_e = 30.049403 

# egg development as a function of time
def egg_dev(t):
    egg = gamma_e*np.exp(-(temp(t)-beta_e)**2/alpha_e)
    return(egg)

# Fit scalars in larvae development rate function
alpha_l = 109.03255
beta_l = 20.18118
gamma_l = 19.56231

# larva development as a function of time
def larva_dev(t):
    larva =  gamma_l*np.exp(-(temp(t)-beta_l)**2/alpha_l)
    return(larva)


# Fit scalars in pupal development rate function
alpha_p = 329.6702830
beta_p = 39.9070020
gamma_p =  30.5920232

# This function defines pupa development as a function of time
def pupa_dev(t):
    pupa =  gamma_p*np.exp(-(temp(t)-beta_p)**2/alpha_p)
    return(pupa)



##############################################################

alpha_b = 100    # avarage number of eggs laid per oviposition (day^-1) (source: CDC)
phi = gen_cycle(t) #1/4 #0.051      # oviposition rate per adult mosquito (source: https://www.pascomosquito.org/resources/mosquito-biology/, https://megacatch.com/mosquito-lifecycle-faqs/)
delta_E = egg_dev(t) #1/2    # hatching rate of eggs into larvae (per day)
delta_L = larva_dev(t) #1/5    # development rate of larvae into pupae (per day)
delta_P = pupa_dev(t) #1/2    # development rate of pupae into adult (per day)
k = 1/2          # fraction of juviniles becoming adult females
mu_E = 0.01      # egg mortality rate
mu_L = 0.02      # larva mortality rate
mu_P = 0.01      # pupal mortality rate
mu_A = 1/42   # mortality rate of adult mosquitoes (source: https://www.vdci.net/mosquito-biology-101-life-cycle/)

In [3]:
#The Gillespie Algorithm
# This stochastic process is a Continuous Time Markov Chain.

while t[-1] < tend:

        rates = [ alpha_b * phi * A[-1], delta_E * E[-1], delta_L * L[-1], k * delta_P * P[-1], \
                mu_E * E[-1], mu_L * L[-1], mu_P * P[-1], mu_A * A[-1]]

        rate_sum = sum(rates)
        if rate_sum == 0:
                break

        tau = np.random.exponential(scale=1/rate_sum)

        t.append(t[-1] + tau)

        rand = random.uniform(0,1)
        
        # Egg oviposition event
        if rand * rate_sum <= rates[0]:
            
                E.append(E[-1] + 1)
                L.append(L[-1])
                P.append(P[-1])
                A.append(A[-1])
        
        # Hatching of eggs and development of larvae event
        elif rand * rate_sum > rates[0] and rand * rate_sum <= sum(rates[:2]):
            
                E.append(E[-1] - 1)
                L.append(L[-1] + 1)
                P.append(P[-1])
                A.append(A[-1])
                
        # Development of larvae into pupae event
        elif rand * rate_sum > sum(rates[:2]) and rand * rate_sum <= sum(rates[:3]):

                E.append(E[-1])
                L.append(L[-1] - 1)
                P.append(P[-1] + 1)
                A.append(A[-1])

        # Development of pupa into adults event
        elif rand * rate_sum > sum(rates[:3]) and rand * rate_sum <= sum(rates[:4]):

                E.append(E[-1])
                L.append(L[-1])
                P.append(P[-1] - 1)
                A.append(A[-1] + 1)

        # Egg mortality event
        elif rand * rate_sum > sum(rates[:4]) and rand * rate_sum <= sum(rates[:5]):

                E.append(E[-1] - 1)
                L.append(L[-1])
                P.append(P[-1])
                A.append(A[-1])

        # Larvae mortality event
        elif rand * rate_sum > sum(rates[:5]) and rand * rate_sum <= sum(rates[:6]):
            
                E.append(E[-1])
                L.append(L[-1] - 1)
                P.append(P[-1])
                A.append(A[-1])
        
        
        # Pupae mortality event
        elif rand * rate_sum > sum(rates[:6]) and rand * rate_sum <= sum(rates[:7]):

                E.append(E[-1])
                L.append(L[-1])
                P.append(P[-1] - 1)
                A.append(A[-1])

        # Adult mortality event
        elif rand * rate_sum > sum(rates[:7]) and rand * rate_sum <= sum(rates[:8]):
            
                E.append(E[-1])
                L.append(L[-1])
                P.append(P[-1])
                A.append(A[-1] - 1)


In [ ]:
E_plot, = plt.plot(t,E, label="E")
L_plot, = plt.plot(t,L, label="L")
P_plot, = plt.plot(t,P, label="P")
A_plot, = plt.plot(t,A, label="A")

plt.legend(handles=[E_plot, L_plot, P_plot, A_plot])
plt.xlabel("Time")
plt.ylabel("Abundance")
plt.show()

In [ ]:
# #Plotting the data from the simulations
# f, (ax1, ax2, ax3, ax4) = plt.subplots(4, sharex=True, sharey=False)
# line1, = ax1.plot(t , E, color="b",label="Egg")
# line2, = ax2.plot(t , L, color="r",label="Larvae")
# line3, = ax3.plot(t , P, color="y",label="Pupae")
# line4, = ax4.plot(t , A, color="g",label="Adult")
# ax1.set_ylabel('Population')
# ax1.set_xlabel('Time')
# ax1.legend(handles=[line1,line2,line3, line4])
# plt.show()

In [ ]:
# ## ODE (deterministic) plot sanity check

# #Time to end taken from the ssa 
# new_tend = t[-1]

# # [ E, L, P, A ]

# y0 = [5, 0, 0, 0]

# t = np.linspace(0,new_tend,num=1000)




# params = [alpha_b, phi, delta_E, delta_L, delta_P, k, mu_E, mu_L, mu_P, mu_A]

# def sim(variables,t,params):

#     E = variables[0]
#     L = variables[1]
#     P = variables[2]
#     A = variables[3]
    
#     alpha_b = params[0]
#     phi = params[1]
#     delta_E = params[2]
#     delta_L = params[3]
#     delta_P = params[4]
#     k = params[5]
#     mu_E = params[6]
#     mu_L = params[7]
#     mu_P = params[8]
#     mu_A = params[9]
    

#     dEdt = alpha_b * phi * A - (delta_E + mu_E) * E

#     dLdt = delta_E * E - (delta_L + mu_L) * L

#     dPdt = delta_L * L - (delta_P + mu_P) * P

#     dAdt = k * delta_P * P - mu_A * A



#     return([dEdt, dLdt, dPdt, dAdt])


# y = odeint(sim,y0, t, args=(params,))


# plt.plot(t,y[:,0]) # E
# plt.plot(t,y[:,1]) # L
# plt.plot(t,y[:,2]) # P
# plt.plot(t,y[:,3]) # A
# plt.xlabel("Time")
# plt.ylabel("Abundance")
# plt.show()